In [2]:
import numpy as np
import pandas as pd
import os
import glob
from astropy.io import fits
import emcee
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from astropy.time import Time

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set_context('paper')

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
df = pd.read_csv('../data/io_preliminary_results.csv')

In [4]:
df

,brightness,brightness_unc,gaussian_width,gaussian_width_unc,int_wv,obs_line_center,obs_line_center_unc
0,-2.103751,0.245665,0.104852,0.014980,1208.850,1208.987646,0.006165
1,12.200903,6.988241,0.065915,0.030816,1211.212,1210.969454,0.612868
2,66.134837,3540.150051,0.054823,0.001590,1211.380,1211.370112,0.002631
